<a href="https://colab.research.google.com/github/KoMurase/interest/blob/master/Fashion_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from torchvision.datasets import FashionMNIST 
from torchvision import transforms

from torch.utils.data import DataLoader,Dataset,TensorDataset
import tqdm
from torch import nn,optim
import torch

In [2]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130


In [3]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-cnoaty4p
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-cnoaty4p
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4307 sha256=16fdf98ee799079b0546a7ff6a71eb29b28d1fa3280587e440bf909e29816dc7
  Stored in directory: /tmp/pip-ephem-wheel-cache-c22o_hmm/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [4]:
!git clone https://github.com/NVIDIA/cuda-samples/
!cp cuda-samples/Common/* /usr/local/include

Cloning into 'cuda-samples'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 1090 (delta 70), reused 120 (delta 67), pack-reused 952
Receiving objects: 100% (1090/1090), 25.39 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (853/853), done.
cp: -r not specified; omitting directory 'cuda-samples/Common/FreeImage'
cp: -r not specified; omitting directory 'cuda-samples/Common/UtilNPP'


In [5]:
fashion_mnist_train = FashionMNIST('../FashionMNIST',
                                  train=True,download=True,transform=transforms.ToTensor())

0it [00:00, ?it/s]

26427392it [00:04, 5670748.78it/s]                              


Extracting ../FashionMNIST/FashionMNIST/raw/train-images-idx3-ubyte.gz


0it [00:00, ?it/s]

32768it [00:00, 34732.86it/s]                           
0it [00:00, ?it/s]

Extracting ../FashionMNIST/FashionMNIST/raw/train-labels-idx1-ubyte.gz


4423680it [00:02, 1653852.80it/s]                            
0it [00:00, ?it/s]

Extracting ../FashionMNIST/FashionMNIST/raw/t10k-images-idx3-ubyte.gz


8192it [00:00, 13928.59it/s]            

Extracting ../FashionMNIST/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


In [0]:
fashion_mnist_test = FashionMNIST('../FashionMNIST',
                                  train=False,download=True,transform=transforms.ToTensor())

In [0]:
batch_size = 128
train_loader = DataLoader(fashion_mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(fashion_mnist_test, batch_size=batch_size, shuffle=False)

In [0]:
#(N,C,H,W)形式のTensorを(N,C*H*W)に引き延ばす層

class FlattenLayer(nn.Module):
  def forward(self,x):
    sizes = x.size()
    return x.view(sizes[0],-1)

  #5X5のカーネルを使用し最初に32個,次に64個のチャネルを作成する
  #BatchNorm2dは画像形式用のDropout 
  #最後にFlattenLayerを挟む
conv_net = nn.Sequential(
    nn.Conv2d(1,32,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(0.25),
    nn.Conv2d(32,64,5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    FlattenLayer()
)

In [0]:
test_input = torch.ones(1,1,28,28)
conv_output_size = conv_net(test_input).size()[-1]

#2層のMLP
mlp = nn.Sequential(
  nn.Linear(conv_output_size, 200),
  nn.ReLU(),
  nn.BatchNorm1d(200),
  nn.Linear(200,10)
)
#最終的なCNN
net = nn.Sequential(
  conv_net,
  mlp
)

In [0]:
#評価のヘルパー関数
def eval_net(net, data_loader,device='cpu'):
  #DropoutやBatchNormを無効化
  net.eval()
  ys = []
  ypreds = []
  
  for x,y in data_loader: 
    x = x.to(device)
    y = y.to(device)
    
    with torch.no_grad():
      _,y_pred = net(x).max(1)
    ys.append(y)
    ypreds.append(y_pred)
    
    ys = torch.cat(ys)
    ypreds = torch.cat(ypreds)
    
    acc = (ys == ypreds).float().sum() / len(ys)
    return acc.item()



In [0]:
def train_net(net,train_loader,test_loader,optimizer_cls=optim.Adam,
             loss_fn=nn.CrossEntropyLoss(),n_iter=10,device='cuda:0'):
  train_losses = []
  train_acc = []
  val_acc = []
  optimizer = optimizer_cls(net.parameters())
  
  for epoch in range(n_iter):
    running_loss = 0.0
    
    #ネットワークを訓練モードにする
    net.train() 
    n = 0
    n_acc = 0
    
    for i , (xx,yy) in tqdm.tqdm(enumerate(train_loader),total=len(train_loader)):
      xx = xx.to(device)
      yy = yy.to(device)
      h = net(xx)
      loss = loss_fn(h,yy)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      running_loss += loss.item() 
      n += len(xx)
      
      _,y_pred = h.max(1)
      n_acc += (yy == y_pred).float().sum().item()
  
  train_losses.append(running_loss / i)
  
  #訓練データの予測精度
  train_acc.append(n_acc / n)
  val_acc.append(eval_net(net,test_loader,device))
  #このエポックでの結果を表示
  print(epoch,train_losses[-1],train_acc[-1],val_acc[-1],flush=True)
  

In [14]:
#ネットワークの全パラメータをGPUに転送
net.to('cuda:0')

#訓練の実行
train_net(net,train_loader,test_loader,n_iter=20,device='cuda:0')

100%|██████████| 469/469 [00:12<00:00, 37.88it/s]

19 0.066478239145512 0.9757666666666667 0.8984375
